<a href="https://colab.research.google.com/github/kwb425/class-2025-spring/blob/main/class-2025-spring_0328-0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers gradio accelerate
!pip install -U bitsandbytes

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import gradio as gr

# Model identifier
model_id = "HuggingFaceH4/zephyr-7b-alpha"

# Load tokenizer and model with 8-bit quantization to reduce memory usage
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_8bit=True,  # Use 8-bit quantization
    trust_remote_code=True
)

# Create text generation pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define chat function
def chat(prompt):
    # Format the prompt using the chat template
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    response = pipe(formatted_prompt, max_new_tokens=256, do_sample=True, temperature=0.7)
    return response[0]['generated_text'].split("<|assistant|>")[-1].strip()

# Launch Gradio interface
gr.Interface(fn=chat, inputs="text", outputs="text", title="Zephyr-7B-α Chatbot").launch()